# iSWAP 门

*版权所有 (c) 2021 百度量子计算研究所，保留所有权利。*

## 内容概要
本教程将演示如何使用量脉生成 iSWAP 门的优化脉冲，本教程的大纲如下：
- 背景介绍
- 准备工作
- 构造哈密顿量
- 通过量脉云服务生成与优化脉冲
- 总结

## 背景介绍
iSWAP 门是超导量子计算中的一个原生门，因为它可以通过两个超导量子比特之间的耦合（XY 相互作用）直接实现。在超导电路中，iSWAP 门是通过将两个量子比特调谐到共振，并持续一段时间来实现的。iSWAP 门在两个量子比特之间交换激发态，并增加一个 $i$ 的相位 \[1\]。iSWAP 门的酉矩阵为：
$$
\rm iSWAP = \begin{pmatrix}
    1 & 0 & 0 & 0 \\
    0 & 0 & -i & 0 \\
    0 & -i & 0 & 0 \\
    0 & 0 & 0 & 1   
\end{pmatrix}.
$$
通过旋转波近似，量子比特间的耦合哈密顿量可以写成 \[2\]：
$$
\hat H_{\rm coup} = g(e^{i\delta_{01}t}\hat{a}_0^\dagger\hat{a}_1+e^{-i\delta_{01}t}\hat{a}_0\hat{a}_1^\dagger),
$$
其中 $\delta_{12}=\omega_{q1}-\omega_{q2}$ 是两个量子比特间的失调（$\omega _{qi}$ 是第 $i$ 个量子比特的频率）。通过调节量子比特 1 的磁通，使其与量子比特 2 共振，失调 $\delta_{12}=0$。 $\sigma_i^+$ 和 $\sigma_i^-$是第 $i$ 个 qubit 的产生和湮灭算符。耦合哈密顿量简化为：
$$
\hat H_{\rm coup} = g(\hat{a}_0^\dagger\hat{a}_1+\hat{a}_0\hat{\sigma}_1^\dagger),
$$
从表达式可以看出，这一项导致两个量子比特之间激发态的交换。

此外，我们可以使用两个 iSWAP 门和几个单量子比特门来生成 CNOT 门。实验上，我们还可以利用 iSWAP 门产生 GHZ 态 \[2\]。

## 准备工作
成功安装量脉后，您可以按照本教程运行下面的量脉程序。要运行此教程，您需要从 Quanlse 和其它常用的 Python 库导入以下的包：

In [ ]:
# Import numpy and math constants
from numpy import round
from math import pi

# Import the Hamiltonian module
from Quanlse.QHamiltonian import QHamiltonian as QHam
from Quanlse.QOperator import duff, number

# Import Optimizer for ISWAP gate
from Quanlse.remoteOptimizer import remoteOptimizeISWAP

# Import tools for analysing results
from Quanlse.Utils.Functions import project

## 构造哈密顿量

使用量脉优化前，我们需要指定系统的哈密顿量。我们考虑引入每个量子比特的三个最低能级，其中第三能级代表了能级的泄漏。在旋转坐标系中，两个耦合量子比特的系统哈密顿量表示为 \[1\]：
$$
\hat H =(\omega_{q1}-\omega_{d1})\hat a^\dagger_1 \hat a_1+(\omega_{q2}-\omega_{d2})\hat a^\dagger_2 \hat a_2+ \frac{\alpha _1}{2}\hat a^\dagger_1 \hat a^\dagger_1 \hat a_1 \hat a_1 + \frac{\alpha_2}{2}\hat a^\dagger_2 \hat a^\dagger_2 \hat a_2 \hat a_2 +\frac{g}{2}(\hat a_1\hat a_2^\dagger+\hat a_1^\dagger \hat a_2) + \sum_i\frac{A^z_i(t)}{2}\hat a^\dagger_i \hat a_i,
$$
其中 $\hat a_i^\dagger$ ($\hat a_i$) 是第 $i$（$i=1,2$）个量子比特的产生（湮灭）算符；$\omega _{qi}$ 是第 $i$ 个量子比特的频率；$\alpha_i$ 是第 $i$ 个量子比特的失谐；$g$ 是两个量子比特的耦合强度。$A_i^z(t)$ 是施加到第 $i$ 个量子比特的磁通。

量脉通过指定采样周期、门时间以及运行计算所需的其它参数来定义系统哈密顿量。量脉可以灵活地为任意量子门操作进行定制模拟，特别是那些涉及参数优化的过程。首先，我们定义几个参数来设置采样周期、量子比特数和系统的维度：

In [ ]:
# Sampling period
dt = 1.0
# number of qubits
qubits = 2
# System energy level
level = 3

然后我们定义两个量子比特的频率、驱动频率和失谐性以及它们之间的耦合强度：

In [ ]:
qubitArgs = {
    "coupling": 0.0277 * (2 * pi),  # Coupling of Q0 and Q1
    "qubit_freq0": 5.805 * (2 * pi),  # Frequency of Q0
    "qubit_freq1": 5.205 * (2 * pi),  # Frequency of Q1
    "drive_freq0": 5.205 * (2 * pi),  # Drive frequency on Q0
    "drive_freq1": 5.205 * (2 * pi),  # Drive frequency on Q1
    "qubit_anharm0": -0.217 * (2 * pi),  # Anharmonicity of Q0
    "qubit_anharm1": -0.226 * (2 * pi)  # Anharmonicity of Q1
}

接下来，我们使用 `QHamiltonian()` 函数创建一个空的哈密顿量的字典，并根据先前定义的参数向其中添加漂移项与耦合项：
$$
\begin{align}
\hat H_{\rm drift} &= (\omega_{q0}-\omega_{d0})(\hat a^\dagger_0 \hat a_0)+(\omega_{q1}-\omega_{d1})(\hat a^\dagger_1 \hat a_1)+\frac{\alpha _0}{2}\hat a^\dagger_0 \hat a^\dagger_0 \hat a_0 \hat a_0 + \frac{\alpha_1}{2}\hat a^\dagger_1 \hat a^\dagger_1 \hat a_1 \hat a_1, \\
\hat H_{\rm coup} &= \frac{g}{2}(\hat a_0\hat a_1^\dagger+\hat a_0^\dagger \hat a_1).
\end{align}
$$

在量脉的 `Operator` 模块中，我们提供了一些允许用户快速构造常用算符的工具。失谐项 $(\omega_{\rm q}-\omega_{\rm d})\hat{a}^\dagger\hat{a}$ 和非谐项 $\frac{\alpha}{2}\hat{a}^\dagger\hat{a}^\dagger\hat{a}\hat{a}$ 可以分别使用 `Operator` 模块中的 `number(n)` 和 `duff(n)` 生成：这两个函数 `number(n)` 和 `duff(n)` 分别返回 $n\times n$ 的粒子数算符和 Duffing 算符。耦合项的形式为 $\frac{g}{2}(\hat{a}_i^\dagger\hat{a}_j+\hat{a}_i\hat{a}_j^\dagger)$，可以使用函数 `addCoupling()` 直接添加到哈密顿量中。

In [ ]:
# Create the Hamiltonian
ham = QHam(qubits, level, dt)

# Add the drift term(s)
for qu in range(2):
    # Add the detuning term(s)
    ham.addDrift(number(level), qu, (qubitArgs[f"qubit_freq{qu}"] - qubitArgs[f"drive_freq{qu}"]))
    # Add the anharmonicity term(s)
    ham.addDrift(duff(level), qu, qubitArgs[f"qubit_anharm{qu}"] / 2)

# Add coupling term
ham.addCoupling([0, 1], qubitArgs["coupling"] / 2)

需要注意的是，量脉的优化函数会自动添加控制项：

$$ \hat{H}_{\rm ctrl} = \frac{A_0^z(t)}{2}\hat{a}_0^\dagger \hat{a}_0, $$

所以我们不需要手动添加这项。

所有关于系统哈密顿量的有用信息都存储在 `ham` 中。在开始优化之前，通常我们会为振幅设置一个边界。您可能需要多次运行优化并逐渐缩小边界。

In [ ]:
aBound = (-5, 5)

## 通过量脉云服务生成与优化脉冲

在使用量脉云服务之前，我们需要一个 token 来访问云端。

In [ ]:
# Import tools to get access to cloud service
from Quanlse import Define

# To use remoteOptimizerISWAP() on cloud, paste your token (a string) here
Define.hubToken = ''

基于前面的准备工作，我们接下来开始优化脉冲。函数 `remoteOptimizeISWAP()` 需要传入五个参数。其中，`tg` 指定门持续时间；当失真度小于 `targetInfidelity` 或迭代次数超过 `maxIter` 时，优化将终止并返回最小失真度和具有优化控制项的系统哈密顿量。我们可以通过调用 `qham.printWaves()` 绘制波形，并调用 `qham.getUnitary()` 获取演化酉矩阵。

在本教程中，作为性能评估指标的门失真度被定义为 ${\rm infid} = 1 - \frac{1}{d}\left|{\rm Tr}
[U^\dagger_{\rm goal}P(U)]\right|$，其中 $d$ 是 $U_{\rm goal}$ $(U_{\rm goal} = {\rm iSWAP})$ 的维度，$U$ 是三能级系统的酉演化。值得注意的是，$P(U)$ 是描述被投影到计算子空间的演化。我们首先运行该优化，然后计算投影演化 $P(U)$：

In [ ]:
# Run the optimization
gateJob, infidelity = remoteOptimizeISWAP(ham, aBound=aBound, tg=40, maxIter=3, targetInfidelity=0.005)

# Print infidelity and the waveforms
print(f"minimum infidelity: {infidelity}")
gateJob.plot()

# Print the projected evolution
result = ham.simulate(job=gateJob)
print("The projected evolution P(U):\n", round(project(result[0]["unitary"], qubits, level, 2), 2))

我们可以看到脉冲的驱动强度在 3.8 左右。我们可以通过将边界缩小到（-4.0，-3.6）来搜索更好的保真度。

In [ ]:
aBound = (-4.0, -3.6)
# Run the optimization
gateJob, infidelity = remoteOptimizeISWAP(ham, aBound, tg=40, maxIter=5, targetInfidelity=0.01)

# Print infidelity and the waveforms
print(f"minimum infidelity: {infidelity}")
gateJob.plot()

# Print the projected evolution 
result = ham.simulate(job=gateJob)
print("The projected evolution P(U):\n", round(project(result[0]["unitary"], qubits, level, 2), 2))

如果希望进一步降低失真度，请尝试增大参数 `maxIter`。当用户提供一组合理的参数时，量脉通常可以找到高保真度的优化波形。

## 总结
本教程使用量脉对 iSWAP 门的脉冲进行优化。我们鼓励用户尝试不同于本教程的参数值以获得最佳结果。

## 参考文献

\[1\] [Schuch, Norbert, and Jens Siewert. "Natural two-qubit gate for quantum computation using the XY interaction." *Physical Review A* 67.3 (2003): 032301.](https://link.aps.org/doi/10.1103/PhysRevA.67.032301)

\[2\] [Krantz, Philip, et al. "A quantum engineer's guide to superconducting qubits." *Applied Physics Reviews* 6.2 (2019): 021318.](https://doi.org/10.1063/1.5089550)